# GPUインスタンスを起動する

S3に置かれている一連の画像ファイルを [OpenPose](https://github.com/CMU-Perceptual-Computing-Lab/openpose) で処理するためのGPUインスタンスを起動します。

## VCCアクセストークンの入力

AWSのGPUインスタンスを VCノードとして起動するので、VC Controllerのアクセストークンが必要となります。次のセルを実行すると入力枠が表示されるのでアクセストークンの値を入力してください。

In [ ]:
from getpass import getpass
vcc_access_token = getpass()

アクセストークンの値が正しいことを確認します。

In [ ]:
from common import logsetting
from vcpsdk.vcpsdk import VcpSDK

vcp = VcpSDK(vcc_access_token)

In [ ]:
vcp.df_ugroups()

# VCノードを起動する

AWSのGPUインスタンスを１ノード、VCノードとして起動します。

## VCノードのspecを指定する

GPUを利用するVCノードのspecを指定します。

In [ ]:
spec = vcp.get_spec('aws', 'gpu')
spec.image = 'vcp/gpu:1.4.2-nvidia-384'
spec.cloud_image = 'niivcp-gpu-20180903'
spec.ip_addresses = ['172.30.2.150']

In [ ]:
spec.instance_type = 'g3.4xlarge'     # 1GPU
# spec.instance_type = 'g3.8xlarge'   # 2GPU

VCノードに登録するSSHの公開鍵を指定します。

In [ ]:
import os
spec.set_ssh_pubkey(os.path.expanduser('~/.ssh/id_rsa.pub'))

`spec`の設定内容を確認します。

In [ ]:
print(spec)

## VCノードの起動

In [ ]:
ug = vcp.create_ugroup('openpose-05')

In [ ]:
ug.create_unit('gpu', spec)

起動したVCノードの状態を表示します。

In [ ]:
ug.df_nodes()

## Ansibleの設定

VCノードをAnsibleのインベントリに登録します。

In [ ]:
from pathlib import Path

inventory = Path('./hosts')
if inventory.exists():
    inventory.rename(inventory.with_suffix('.orig'))

with inventory.open(mode='w') as f:
    for x in ug.find_ip_addresses():
        f.write(f'[{ug.name}]\n')
        f.write(f'{x}\n')
    f.write(f'''
[{ug.name}:vars]
ansible_user=root

''')
    
!cat {inventory}
if inventory.with_suffix('.orig').exists():
    try:
        !diff -u {inventory.with_suffix('.orig')} {inventory}
    except:
        pass

`~/.ssh/known_hosts`を更新します。

In [ ]:
for addr in ug.find_ip_addresses():
    !ssh-keygen -R {addr}
    !ssh-keyscan -H {addr} >> ~/.ssh/known_hosts

In [ ]:
ansible_cfg = Path('./ansible.cfg')
if ansible_cfg.exists():
    ansible_cfg.rename(ansible_cfg.with_suffix('.orig'))

with ansible_cfg.open(mode='w') as f:
    f.write(f'''
[defaults]
inventory = {inventory.absolute()}
''')
    
!cat {ansible_cfg}
if ansible_cfg.with_suffix('.orig').exists():
    try:
        !diff -u {ansible_cfg.with_suffix('.orig')} {ansible_cfg}
    except:
        pass

Ansibleでの疎通確認を行います。

In [ ]:
target = ug.name

In [ ]:
!ansible {target} -m ping

In [ ]:
!ansible {target} -b -a 'whoami'

## GPU環境の確認

`nvidia-smi`コマンドを実行して GPU 環境の確認を行います。

In [ ]:
!ansible {target} -a 'docker pull nvidia/cuda:8.0-cudnn5-runtime-ubuntu16.04'

In [ ]:
!ansible {target} -a 'docker run --runtime=nvidia --rm nvidia/cuda:8.0-cudnn5-runtime-ubuntu16.04 \
    nvidia-smi'

## ツールなどのインストール

`docker-compose` をインストールします。

In [ ]:
!ansible {target} -b -m shell \
    -a 'curl -L "https://github.com/docker/compose/releases/download/1.23.2/docker-compose-$(uname -s)-$(uname -m)" \
    -o /usr/local/bin/docker-compose warn=False'

In [ ]:
!ansible {target} -b -m file -a 'path=/usr/local/bin/docker-compose mode=0755'

`docker-compose`を実行できることを確認します。

In [ ]:
!ansible {target} -a 'docker-compose version'

S3へのアクセスに[goofys](https://github.com/kahing/goofys)を利用する際に必要となる fuse をインストールします。

In [ ]:
!ansible {target} -b -m apt -a 'name=fuse update_cache=yes'

`/dev/fuse`があることを確認します。

In [ ]:
!ansible {target} -a 'ls -l /dev/fuse'